# Image_Classification_Pytorch
https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
    -  [檢查CSV檔缺失值](#4.2)
1. [定義模型方法](#5)
1. [定義回調函數方法](#6)
1. [製作資料集＆資料擴增&訓練模型](#7)
1. [混淆矩陣](#8)
1. [提交](#9)
1. [待辦事項](#10)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
# 資料處理套件
import os
import gc
import sys
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from albumentations import *

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus'] = False # 用來正常顯示負號

In [ ]:
# pytorch深度學習模組套件
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset,DataLoader

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = False

# (Boolean)是否為 Colab
COLAB = False


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    OUTPUT_PATH = r'/kaggle/working/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'datasets/AI_CUP_2020_AIMango_Defective_Classification/' 

# (String)訓練資料路徑
TRAIN_DATA_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/Train'

# (String)訓練CSV路徑，如為None則不讀CSV檔
TRAIN_CSV_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/train.csv'

# (String)測試資料路徑
TEST_DATA_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/Test'

# (String)測試CSV路徑，如為None則不讀CSV檔
TEST_CSV_PATH = DATA_ROOT_PATH+r'test_example.csv'

# (String)專案名稱
PROJECT_NAME = 'AI_CUP_2020_AIMango_Defective_Classification'

# (String)專案檔案儲存路徑
if LOCAL or COLAB:
    OUTPUT_PATH = PATH
PROJECT_PATH = OUTPUT_PATH+PROJECT_NAME+'/'+PROJECT_NAME+' '+datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

# (String)權重名稱(使用哪個權重)
WEIGHTS_NAME = 'efficientnetb7'

# (String)模型名稱(使用哪個模型)
MODEL_NAME = 'efficientnetb7'

# (String)讀取預訓練權重的儲存路徑 
LOAD_WEIGHTS_PATH = PROJECT_PATH+r'/models/backup/'+WEIGHTS_NAME+'.pth'

# (String)讀取預訓練模型的儲存路徑 
LOAD_MODEL_PATH = PROJECT_PATH+r'/models/backup/'+MODEL_NAME+'.pth'

# (String)訓練模型的儲存路徑
TRAIN_MODEL_PATH = PROJECT_PATH+r'/models/'+MODEL_NAME+'.pth'

In [ ]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if DEVICE != "CPU":
    !nvidia-smi

In [ ]:
if os.path.isfile(TRAIN_CSV_PATH) and os.path.isfile(TEST_CSV_PATH):
    LOAD_CSV = True
else:
    LOAD_CSV = False

In [ ]:
if not os.path.isdir(PROJECT_PATH+r'/models/'):
    os.makedirs(PROJECT_PATH+r'/models/')

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)分類數量
CLASSES = 3

# (Int)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = 1

# (Int)沒CSV檔，FOLD該參數固定為1
if not LOAD_CSV:
    FOLD = 1
    
# (Int)圖片尺寸
IMAGE_SIZE = [224]*FOLD

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.jpg'

# (String)CSV圖片檔名欄位(不包含路徑)
IMAGE_NAME = 'image_id'

# (String)CSV圖片檔名欄位(包含路徑)
IMAGE_NAME_ROOT = 'image'

# (String)CSV標籤欄位
LABEL_NAME = 'grade'

# (String)CSV標籤欄位類型
LABEL_NAME_TYPE = 'string'

# (Boolean)CSV圖片檔名欄位是否包含副檔名
IMAGE_NAME_HAVE_EXTENSION = True

# (Int)不包含副檔名的圖片檔名長度，因為CSV檔名欄位有副檔名時需要移除
IMAGE_NAME_LENGTH = 5

# (String)測試集CSV標籤欄位
TEST_LABEL_NAME = 'label'

# (String)測試集CSV儲存檔名
TEST_CSV_FILE_NAME = 'submission.csv'

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

# (Boolean)如為True每次返回的卷積算法將是確定的，即默認算法
CUDNN_DETERMINISTIC = True

# (Boolean)PyTorch 中對模型裡的卷積層進行預先的優化，也就是在每一個卷積層中測試 cuDNN 提供的所有卷積實現算法，
# 然後選擇最快的那個。這樣在模型啟動的時候，只要額外多花一點點預處理時間，就可以較大幅度地減少訓練時間
CUDNN_BENCHMARK = True


'''資料擴增參數設定'''


''''模型參數設定'''

# (Boolean)是否印出完整模型
MODEL_PRINT = True


''''回調函數參數設定'''


''''編譯參數設定'''

# (Float)優化器學習率 
LEARNING_RATE = 1e-3

# (String)優化器指定，None為客制，須另外撰寫
BASE_OPTIMIZERS = optim.Adam

# (String)損失函數，None為客制，須另外撰寫
BASE_LOSSES = nn.CrossEntropyLoss

# (String)指定還原成適用於輸出，預設mean
REDUCTION = "mean"


''''訓練參數設定'''

# (Int List)每批訓練的尺寸
BATCH_SIZE = [16]*FOLD

# (Int)使用基於進程的線程時，要啟動的最大進程數。如果未指定，NUM_WORKERS則默認為1。
NUM_WORKERS = 1

# (Boolean)批次處理在大小不合適的情況下，是否刪除最後一個不完整的批次
DROP_LAST = False


''''圖表參數設定'''

# (Float)全部SNS圖表的字形縮放
ALL_SNS_FONT_SCALE = 1.0

# (Int)CSV缺失值圖表寬度
CSV_COUNTPLOT_FIGSIZE_W = 10

# (Int)CSV缺失值圖表高度
CSV_COUNTPLOT_FIGSIZE_H = 10

# (Int)CSV缺失值圖表標題字型大小
CSV_COUNTPLOT_TITLE_FONTSIZE = 20

# (Int)CSV缺失值圖表X軸標題字型大小
CSV_COUNTPLOT_XLABEL_FONTSIZE = 15

# (Int)CSV缺失值圖表Y軸標題字型大小
CSV_COUNTPLOT_YLABEL_FONTSIZE = 15

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = CUDNN_DETERMINISTIC
    torch.backends.cudnn.benchmark = CUDNN_BENCHMARK

seed_everything(SEED)

In [ ]:
# 設置sns圖表縮放係數
sns.set(font_scale = ALL_SNS_FONT_SCALE)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
if LOAD_CSV:
    print('Reading data...')

    # 讀取訓練資料集CSV檔
    train_csv = pd.read_csv(TRAIN_CSV_PATH,encoding="utf8")

    # 讀取測試資料集CSV檔
    test_csv = pd.read_csv(TEST_CSV_PATH,encoding="utf8")

    print('Reading data completed')

In [ ]:
if LOAD_CSV:
    # CSV檔名欄位包括副檔名時，需要移除副檔名
    if IMAGE_NAME_HAVE_EXTENSION:
        train_csv[IMAGE_NAME] = train_csv[IMAGE_NAME].str.slice(stop = IMAGE_NAME_LENGTH)
        test_csv[IMAGE_NAME] = test_csv[IMAGE_NAME].str.slice(stop = IMAGE_NAME_LENGTH)

In [ ]:
if LOAD_CSV:
    # 顯示訓練資料集CSV檔
    print(train_csv.head())

In [ ]:
if LOAD_CSV:
    print("Shape of train_data :", train_csv.shape)

In [ ]:
if LOAD_CSV:
    # 顯示測試資料集CSV檔
    print(test_csv.head())

In [ ]:
if LOAD_CSV:
    print("Shape of test_data :", test_csv.shape)

## 4.2 檢查CSV檔缺失值 <a class="anchor" id="4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
if LOAD_CSV:
    total = train_csv.isnull().sum().sort_values(ascending = False)
    percent = (train_csv.isnull().sum()/train_csv.isnull().count()*100).sort_values(ascending = False)
    missing_train_csv  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_train_csv.head())

In [ ]:
if LOAD_CSV:
    print(train_csv[LABEL_NAME].value_counts())
    f,ax = plt.subplots(figsize=(CSV_COUNTPLOT_FIGSIZE_W, CSV_COUNTPLOT_FIGSIZE_H))
    sns.countplot(train_csv[LABEL_NAME], hue = train_csv[LABEL_NAME],ax = ax)
    plt.title("LABEL COUNT", fontsize=CSV_COUNTPLOT_TITLE_FONTSIZE)
    plt.xlabel(LABEL_NAME.upper(), fontsize=CSV_COUNTPLOT_XLABEL_FONTSIZE)
    plt.ylabel("COUNT", fontsize=CSV_COUNTPLOT_YLABEL_FONTSIZE)
    plt.legend()
    plt.show()

# 5. 定義模型方法<a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

In [ ]:
def build_optimizers():
    if BASE_OPTIMIZERS == None:
        print("Custiom OPTIMIZERS")
    else:
        RETURN_OPTIMIZERS = BASE_OPTIMIZERS
    return RETURN_OPTIMIZERS

optimizer = build_optimizers()

In [ ]:
def build_losses():
    if BASE_LOSSES == None:
        print("Custiom LOSSES")
    else:
        RETURN_LOSSES = BASE_LOSSES
    return RETURN_LOSSES

loss = build_losses()

In [ ]:
class build_model(nn.Module):

    def __init__(self):
        super(build_model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# ==== INIT MODEL    
model = build_model()
model.to(DEVICE)
optimizer = optimizer(model.parameters(), lr=LEARNING_RATE)

# Later we have to filter the invalid steps.
criterion = loss(reduction=REDUCTION)

if MODEL_PRINT:
    print(model)

# 6. 定義回調函數方法<a class="anchor" id="6"></a>
[Back to Table of Contents](#0)

# 7. 製作資料集＆資料擴增&訓練模型 <a class="anchor" id="7"></a>
[Back to Table of Contents](#0)

In [ ]:
# 待增加資料擴增類別
transform = transforms.Compose(
    [transforms.Resize(size=IMAGE_SIZE[0]), # 縮放
     transforms.ToTensor()])

In [ ]:
if LOAD_CSV:
    class DatasetRetriever(Dataset):
        def __init__(self, csv_path=None, fold=fold, transforms=None):
            ids = pd.read_csv(csv_path).id.values
            
            kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
            ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
            self.image_ids = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]
            self.transforms = transforms

        def __len__(self) -> int:
            return len(self.image_ids)

        def __getitem__(self, index: int):
            image_id = self.image_ids[index]
            image = cv2.imread(f'{TRAIN_DATA_PATH}/{image_id}', cv2.IMREAD_COLOR).copy().astype(np.float32)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
            image /= 255.0
            if self.transforms:
                sample = {'image': image}
                sample = self.transforms(**sample)
                image = sample['image']
            return image, image_id

In [ ]:
# class DatasetRetriever(Dataset):
#     def __init__(self, image_ids, transforms=None):
#         ids = pd.read_csv(LABELS).id.values
#         self.image_ids = image_ids
#         self.transforms = transforms
        
#     def __len__(self) -> int:
#         return self.image_ids.shape[0]

#     def __getitem__(self, index: int):
#         image_id = self.image_ids[index]
#         image = cv2.imread(f'{TRAIN_DATA_PATH}/{image_id}', cv2.IMREAD_COLOR).copy().astype(np.float32)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
#         image /= 255.0
#         if self.transforms:
#             sample = {'image': image}
#             sample = self.transforms(**sample)
#             image = sample['image']
#         return image, image_id

In [ ]:
# dataset = DatasetRetriever(
#     image_ids=np.array([path.split('/')[-1] for path in glob(f'{TRAIN_DATA_PATH}/*'+IMAGE_NAME_EXTENSION)]),
#     transforms=transform
# )

# def collate_fn(batch):
#     return tuple(zip(*batch))

# data_loader = DataLoader(
#     dataset,
#     batch_size=BATCH_SIZE[0],
#     shuffle=True,
#     num_workers=NUM_WORKERS,
#     drop_last=DROP_LAST,
#     collate_fn=collate_fn
# )

In [ ]:
# for epoch in range(2):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

# print('Finished Training')

In [ ]:
# # ==== TRAIN LOOP
# tr_it = iter(train_dataloader)

# progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
# losses_train = []

# for itr in progress_bar:

#     try:
#         data = next(tr_it)
#     except StopIteration:
#         tr_it = iter(train_dataloader)
#         data = next(tr_it)

#     model.train()
#     torch.set_grad_enabled(True)
    
#     # Forward pass
#     inputs = data["image"].to(device)
#     target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
#     targets = data["target_positions"].to(device)
    
#     outputs = model(inputs).reshape(targets.shape)
#     loss = criterion(outputs, targets)

#     # not all the output steps are valid, but we can filter them out from the loss using availabilities
#     loss = loss * target_availabilities
#     loss = loss.mean()

#     # Backward pass
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     losses_train.append(loss.item())

#     if (itr+1) % cfg['train_params']['checkpoint_every_n_steps'] == 0 and not DEBUG:
#         torch.save(model.state_dict(), f'model_state_{itr}.pth')
    
#     progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train[-100:])}")

In [ ]:
# torch.save(model.state_dict(), TRAIN_MODEL_PATH)

# 8. 混淆矩陣<a class="anchor" id="8"></a>
[Back to Table of Contents](#0)

# 9. 提交<a class="anchor" id="9"></a>
[Back to Table of Contents](#0)

In [ ]:
# dataiter = iter(testloader)
# images, labels = dataiter.next()

# # print images
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# model = build_model()
# model.load_state_dict(torch.load(PATH))

In [ ]:
# outputs = model(images)

In [ ]:
# _, predicted = torch.max(outputs, 1)

# print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
#                               for j in range(4)))

In [ ]:
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))

In [ ]:
# class_correct = list(0. for i in range(10))
# class_total = list(0. for i in range(10))
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)
#         c = (predicted == labels).squeeze()
#         for i in range(4):
#             label = labels[i]
#             class_correct[label] += c[i].item()
#             class_total[label] += 1


# for i in range(10):
#     print('Accuracy of %5s : %2d %%' % (
#         classes[i], 100 * class_correct[i] / class_total[i]))